<a href="https://colab.research.google.com/github/Dohy-Lee/Do_it-BERT_GPT/blob/main/To_convert_a_Sentence_into_a_Vector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case=False,
)

In [30]:
from transformers import BertConfig, BertModel
pretrained_model_config=BertConfig.from_pretrained( #BERT모델을 프리트레인할 때 설정했던 내용이 담겨 있음
    "beomi/kcbert-base"
)
model=BertModel.from_pretrained(
    "beomi/kcbert-base",
    config=pretrained_model_config #pretrained_model_config 설정에 따라 모델 전체를 초기화한 뒤, 미리 학습된 kcbert-base 체크포인트를 읽어들임. 만약 체크포인트가 로컬에 저장되어 있지 않다면 웹에서 내려받음
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

In [32]:
sentences=['안녕하세요','하이!']
features=tokenizer(
    sentences,
    max_length=10,
    padding='max_length',
    truncation=True,
)

In [33]:
features

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [34]:
features['input_ids'] #inputs_ids : 2개의 입력 문장 각각에 대해 워드피스 토큰화를 수행한 뒤 이를 토큰 인덱스로 변환한 결과. 문장의 시작인 CLS, 끝인 SEP은 각각 인덱스가 2,3임

[[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]]

In [35]:
features['attention_mask'] #attention_mask : 패딩이 아닌 토큰 1, 패딩인 토큰 0

[[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]

In [36]:
features['token_type_ids'] #token_type_ids : 세그먼트 정보로 지금처럼 각각이 1개의 문자으로 구성됐을 떄는 모두 0이됨

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [37]:
import torch
features={k:torch.tensor(v) for k,v in features.items()} #피처 → 토치 텐서로 변환
outputs=model(**features) # BERT모델 실행(임베딩 계산하기)

In [39]:
outputs.last_hidden_state #BERT 마지막 레이어의 단어 수준 벡터 확인 = 단어 수준 임베딩(리프레젠테이션)

tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
         [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
         [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
         ...,
         [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
         [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
         [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],

        [[ 0.9371, -1.4749,  1.7351,  ..., -0.3426,  0.8050,  0.4031],
         [ 1.6095, -1.7269,  2.7936,  ...,  0.3100, -0.4787, -1.2491],
         [ 0.4861, -0.4569,  0.5712,  ..., -0.1769,  1.1253, -0.2756],
         ...,
         [ 1.2362, -0.6181,  2.0906,  ...,  1.3677,  0.8132, -0.2742],
         [ 0.5409, -0.9652,  1.6237,  ...,  1.2395,  0.9185,  0.1782],
         [ 1.9001, -0.5859,  3.0156,  ...,  1.4967,  0.1924, -0.4448]]],
       grad_fn=<NativeLayerNormBackward0>)

In [41]:
outputs.last_hidden_state.shape #문장 2개에 속한 각각의 토큰(시퀀스 길이 10)이 768차원짜리의 벡터로 변환 → 개체명 인식 과제와 같이 단어별로 수행해야하는 태스크에 활용

torch.Size([2, 10, 768])

In [40]:
outputs.pooler_output #BERT 마지막 레이어의 문서 수준 벡터 확인 = 문장 수준 임베딩(리프레젠테이션)

tensor([[-0.1594,  0.0547,  0.1101,  ...,  0.2684,  0.1596, -0.9828],
        [-0.9221,  0.2969, -0.0110,  ...,  0.4291,  0.0311, -0.9955]],
       grad_fn=<TanhBackward0>)

In [42]:
outputs.pooler_output.shape #문장 2개가 각각 768차원짜리의 벡터로 변환 → 문서 분류 과제처럼 문장 전체를 벡터 하나로 변환한 뒤 이 벡터에 어떤 계산을 수행하는 태스크에 활용

torch.Size([2, 768])